<a href="https://colab.research.google.com/github/nicolas1996-ing/agentes-inteligentes/blob/main/sistemas_inteligentes_laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Importaciones**

In [ ]:
%%capture
!pip install -U langchain-google-genai langgraph langchain-core langchain-community

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

**2. Cargar api key Gemini**

In [ ]:
from google.colab import userdata
import os

try:
  GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
  print("API_KEY de Gemini cargada correctamente")
except Exception as e:
  print("Error al cargar API_KEY. Configura en > Runtime > Manage secrets")
  print("Nombre del secreto: GOOGLE_API_KEY")
  raise e

API_KEY de Gemini cargada correctamente


**3. Consultar modelos disponibles**

In [ ]:
from google.generativeai import list_models
import google.generativeai as genai

genai.configure(api_key=GOOGLE_API_KEY)
models = list_models()
print("Modelos disponibles: ")
for model in models:
  print(f"{model.name} | Métodos soportados: {model.supported_generation_methods}")

Modelos disponibles: 
models/embedding-gecko-001 | Métodos soportados: ['embedText', 'countTextTokens']
models/gemini-2.5-flash | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp | Métodos soportados: ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001 | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation | Métodos soportados: ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001 | Métodos soportados: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemin

**4. Definir esquema de estado**

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator

class AgentState(TypedDict):
  messages: Annotated[Sequence[dict], operator.add]
  current_step: str
  analysis_results: dict

**5. Inicializar LLM**

In [ ]:
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    temperature = 0.3,
    google_api_key = GOOGLE_API_KEY
)

**6. Funciones con inyección de dependencias**

In [ ]:
def analyze_input_gemini(state: AgentState, llm_instance):
  """Nodo que analiza la entrada usando Gemini"""
  messages = state['messages']
  gemini_messages = [
      {
          "role": "system",
          "content": "Eres un analista experto. Analiza la solicitud del usuario y determina qué tipo de ayuda necesita."
      },
      *messages
  ]
  response = llm_instance.invoke(gemini_messages)
  print(f"Analisis completado: {response.content[:100]}...")

  return {
      "current_step": "analysis_complete",
      "analysis_results": {
          "intent": "technical_support",
          "confidence": 0.92,
          "details": response.content
      }
  }

def generate_response_gemini(state: AgentState, llm_instance):
  """Nodo que genera una respuesta usando Gemini"""
  messages = state['messages']
  analysis = state['analysis_results']
  system_prompt = f"""
  Eres un asistente técnico experto. El análisis indica:
  - Intención: {analysis['intent']}
  - Confianza: {analysis['confidence']:.2f}
  - Detalles: {analysis['details'][:200]}

  Proporciona una respuesta clara, técnica y útil. Se conciso pero completo.
  """

  gemini_messages = [
      {
          "role": "system",
          "content": system_prompt
      },
      *messages
  ]

  response = llm_instance.invoke(gemini_messages)

  print(f"Respuesta generada: {response.content[:100]}...")

  return {
      "current_step": "response_generated",
      "messages": [
          {
              "role": "assistant",
              "content": system_prompt
          },
          *messages
      ]
  }


**7. Crear wrappers para el grafo**

Esta es la forma estándar de importar clases o funciones específicas de un módulo en Python. Asegúrate de que la biblioteca `langchain-google-genai` esté instalada en tu entorno (usando `!pip install langchain-google-genai`) antes de intentar importarla.

In [ ]:
def analyze_node(state):
  return analyze_input_gemini(state, llm)

def respond_node(state):
  return generate_response_gemini(state, llm)

**8. Construir el grafo**

In [ ]:
workflow = StateGraph(AgentState)
workflow.add_node("analyze", analyze_node)
workflow.add_node("respond", respond_node)
workflow.add_edge(START, "analyze")
workflow.add_edge("analyze", "respond")
workflow.add_edge("respond", END)

**9. Compilar con checkpointing**

In [ ]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

**10. Ejecutar en tiempo real**

In [ ]:
config = { "configurable": { "thread_id": "colab_test_1" } }
initial_state = {
    "messages": [
        {
            "role": "user",
            "content": "¿Qué es un grafo y como se relacion con langgraph?"
        }
    ],
    "current_step": "initial",
    "analysis_results": {}
}

print("Ejecutando agente con Gemini en LangGraph...")
result = app.invoke(initial_state, config=config)

Ejecutando agente con Gemini en LangGraph...
Analisis completado: Eres un experto en IA, y la relación entre grafos y LangGraph es fundamental para entender cómo cons...
Respuesta generada: Un **grafo** es una estructura matemática utilizada para modelar relaciones entre objetos. Consiste ...


**11. Mostrar resultados**

In [ ]:
import json

print("\n" + "="*50)
print("RESULTADO FINAL DEL AGENTE")
print("="*50)
print(f"Paso actual: {result['current_step']}")

print(json.dumps(result['analysis_results'], indent=4, ensure_ascii=False))
print(json.dumps(result['analysis_results'], indent=2, ensure_ascii=False))

print("\n RESPUESTA DEL AGENTE:")
print(result["messages"][-1]["content"])


RESULTADO FINAL DEL AGENTE
Paso actual: response_generated
{
    "intent": "technical_support",
    "confidence": 0.92,
    "details": "Eres un experto en IA, y la relación entre grafos y LangGraph es fundamental para entender cómo construir aplicaciones LLM complejas y robustas. Aquí tienes un análisis detallado:\n\n---\n\n## Análisis: Qué es un Grafo y su Relación con LangGraph\n\n### 1. ¿Qué es un Grafo?\n\nEn ciencias de la computación y matemáticas, un **grafo** es una estructura de datos abstracta que se utiliza para representar un conjunto de objetos y las relaciones entre ellos. Es una de las estructuras de datos más fundamentales y versátiles.\n\nUn grafo consta de dos componentes principales:\n\n1.  **Nodos (o Vértices):** Representan los objetos o entidades individuales. Piensa en ellos como los puntos en un mapa.\n    *   *Ejemplos:* Ciudades, personas en una red social, páginas web, estados en un proceso, componentes de un sistema.\n\n2.  **Aristas (o Bordes):** Represent